In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx

[nltk_data] Downloading package punkt to /Users/pat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
def soup_text_summary(url, n):
    res = requests.get(url)
    html_page = res.content
    
    soup = BeautifulSoup(html_page, 'html.parser')
    text = soup.find_all(text=True)
    
    output = ''
    blacklist = ['[document]','abbr','b','bdi','body','button','caption','cite','div','footer','form','h2','head',
             'header','html','i','input','label','li','main','nav','ol','script','small','span','style', 'td',
             'th','title','tr','ul'
            ]

    for t in text:
        if t.parent.name not in blacklist:
            output += '{} '.format(t)
    
    output = output.replace("\n", '')
    
    sentences = []
    sentences.append(sent_tokenize(output))
    sentences = [y for x in sentences for y in x]
    
    word_embeddings = {}
    f = open('glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()

    clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
    clean_sentences = [s.lower() for s in clean_sentences]
    stop_words = stopwords.words('english')

    def remove_stopwords(sen):
        sen_new = " ".join([i for i in sen if i not in stop_words])
        return sen_new
    
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    
    sentence_vectors = []
    for i in clean_sentences:
        if len(i) != 0:
            v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
        else:
            v = np.zeros((100,))
        sentence_vectors.append(v) 
    
    sim_mat = np.zeros([len(sentences), len(sentences)])

    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]
    
    nx_graph = nx.from_numpy_array(sim_mat)
    scores = nx.pagerank(nx_graph) 
    
    ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
    
    print("TITLE:")
    print(soup.title.get_text())
    print("\n")
    print("SUMMARY:")

    for i in range(n):
        print(ranked_sentences[i][1])
    
    



In [3]:
url = 'https://www.espn.com/mlb/insider/story/_/id/35981176/opening-day-2023-pitch-clock-new-rules-revolutionize-mlb-season'

In [4]:
soup_text_summary(url, 5)

<ipython-input-2-7dfc09a06a08>:6: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = soup.find_all(text=True)
<ipython-input-2-7dfc09a06a08>:33: FutureWarning: The default value of regex will change from True to False in a future version.
  clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")


TITLE:
Opening Day 2023: How the pitch clock will revolutionize MLB


SUMMARY:
People inside the game of baseball love the pitch clock so far.
And he worried that the drama baseball naturally produced -- the downtime between pitches that let the game breathe and infused late innings with tension -- would be lost, and with it would go baseball's soul, the element that makes it different from all the other sports.
Just like everyone else in baseball, the 27-year-old left-hander has time at the forefront of his mind these days.
Betting tips for the 2023 MLB season: futures A beginner's guide to betting the 2023 MLB season The poker game between Brazil, Real Madrid, Spurs, Ancelotti and Pochettino NWSL added VAR to fix referee problems, but its debut only added controversy The real winners and losers from the U.S.'s international break Walking the walk: Derrick James, Anthony Joshua's new trainer, has a special postfight tradition to enjoy success WrestleMania 39: How Cody Rhodes' departur